# Q&A about the document with source

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain/blob/main/examples/text_summarization.ipynb)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain.llms import OpenAI

In [ ]:
with open('/content/sample.txt') as f:
    dragonball_txt = f.read()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_text(dragonball_txt)

In [ ]:
docsearch = FAISS.from_texts(
    texts, 
    embeddings, 
    metadatas=[{"source": i} for i in range(len(texts))])
docs = docsearch.similarity_search(query)

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

# load_qa_with_sources_chainの準備
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

# ソース付きの質問応答の実行
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [ ]:
print(texts[0])
print(texts[1])
print(texts[2])
print(texts[3])